# 句子情感分类

我们的目标是创建一个模型，该模型以句子为输入（就如上述数据集中的评论），输出为1（句子带有积极情感）或者0（句子带有消极情感）。
实际上，该模型是由两个子模型组成的。

DistilBERT模型负责处理句子并将从中提取的信息传给下一个模型，这是由HuggingFace团队开发的开源知识蒸馏版BERT模型，量级更轻、运行更快，性能堪比BERT。

下一个模型是scikit-learn中一个基本的逻辑回归模型，接收DistilBERT处理的结果，并将句子分类为积极或消极（分别为1或0）。

两个模型间传递的数据是一个768维的向量。我们可以把这个向量当作用于分类的句子的嵌入（embedding）。

## 模型训练
虽然我们要用到两个模型，但只需训练逻辑回归模型即可。DistillBERT模型将使用适用于英语语言处理的预训练模型。这种模型没有专门为句子分类任务进行过训练和微调，但是，基于BERT模型的通用目标，它还是具有一定的句子分类能力，尤其是第一位置（与[CLS]标志相关）的BERT输出。我认为，这是由BERT模型的次要训练目标，即下一句分类（Next sentence classification）决定的。这一目标似乎是训练模型去封装整句意思作为第一位置的输出。Transformers库包含DistilBERT模型及其预训练版本模型的实现。

讲解的网址：
https://github.com/jalammar/jalammar.github.io/blob/master/notebooks/bert/A_Visual_Notebook_to_Using_BERT_for_the_First_Time.ipynb

https://mp.weixin.qq.com/s?__biz=MzI4MDYzNzg4Mw==&mid=2247492910&idx=4&sn=4b884e7a72bd5c231f0244274efacb1c&chksm=ebb7ddfadcc054ec4121ed0d5d48b2d14aa9810ebfe3d0d4353f9b3a2b896f9b611f4686e29e&mpshare=1&scene=24&srcid=0418PcAneZb84Lz46CjEFx1a&sharer_sharetime=1587216489039&sharer_shareid=1176345ce55a5e0024df723ca29d3249&key=240d40e83da786d0b11b0fa5f82466301828c99323aa10856d86cb6a8e186878312ed2ef020da3f9997ce589e597bb4f98823056e1b567a43efd9ac3698d5cc30a64df90d379765343f888c3a8592cebeee4def25bded5d82a4db18906f2af1e8a5bbd9f0fde293cf1dc3e2aa187d388c6340d0b86c6989ac08703b80d932c6a&ascene=14&uin=MzM4MTIwMjg2MA%3D%3D&devicetype=Windows+10+x64&version=6300002f&lang=zh_CN&exportkey=A8h%2FCbBMOVM%2FfgZxORy8PZg%3D&pass_ticket=Jk9za4U5mb%2FrD6y79f0yBI%2B2K6ty5t0bYcfkqx0Ev3dHEBcJ74BIdfW00BCTppUR&wx_header=0

In [1]:
# 首先，使用训练后的distilBERT模型来生成数据集中2000个句子的句子嵌入。
# 这一步后就不再用distilBERT，剩下的都是scikit-learn的工作。依照惯例，将数据集划分为训练集和测试集。
# 第一步，使用BERT 分词器将英文单词转化为标准词（token）。第二步，加上句子分类所需的特殊标准词（special token，如在首位的[CLS]和句子结尾的[SEP]）。
# 第三步，分词器会用嵌入表中的id替换每一个标准词（嵌入表是从训练好的模型中得到的），词嵌入的背景知识可参见我的《图解Word2Vec》。
import numpy as np
import pandas as pd
import torch
import transformers as ppb#pytorch transformers
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

F:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
F:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
F:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
F:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is depr

In [5]:
#学习直接从github中导入数据集
#翻墙之后下载下来的成功率更高
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)

In [6]:
df.head()

,0,1
0,"a stirring , funny and finally transporting re...",1
1,apparently reassembled from the cutting room f...,0
2,they presume their audience wo n't sit still f...,0
3,this is a visually stunning rumination on love...,1
4,jonathan parker 's bartleby should have been t...,1


In [8]:
# 导入预训练的DistilBERT模型和分词器
model_class,tokenizer_class,pretrained_weights=(ppb.DistilBertModel,ppb.DistilBertTokenizer,'distilbert-base-uncased')
## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')
# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

#### 现在我们可以对数据集进行分词。请注意，此处的操作与上面的示例有些不同。上面的示例仅进行了一个句子的分词。在这里，我们将分批处理所有句子的分词（考虑到资源问题，ipython notebook文件中只处理了一小部分数据，约2000个）。

In [9]:
# 分词
tokenized=df[0].apply((lambda x:tokenizer.encode(x,add_special_tokens=True)))
tokenized

0       [101, 1037, 18385, 1010, 6057, 1998, 2633, 182...
1       [101, 4593, 2128, 27241, 23931, 2013, 1996, 62...
2       [101, 2027, 3653, 23545, 2037, 4378, 24185, 10...
3       [101, 2023, 2003, 1037, 17453, 14726, 19379, 1...
4       [101, 5655, 6262, 1005, 1055, 12075, 2571, 376...
                              ...                        
6915    [101, 9145, 1010, 7570, 18752, 14116, 1998, 28...
6916    [101, 2202, 2729, 2003, 19957, 2864, 2011, 103...
6917    [101, 1996, 5896, 4472, 4121, 1010, 3082, 7832...
6918    [101, 1037, 5667, 2919, 2143, 2007, 5667, 2561...
6919    [101, 1037, 12090, 2135, 2512, 5054, 19570, 23...
Name: 0, Length: 6920, dtype: object

In [10]:
# len(tokenized[10])
# tokenized
np.array(tokenized).shape

(6920,)

In [11]:
# Padding步骤
max_len=0
for i in tokenized.values:
    if len(i)>max_len:
        max_len=len(i)
padded=np.array([i+[0]*(max_len-len(i)) for i in tokenized.values])

In [12]:
np.array(padded).shape

(6920, 67)

In [15]:
# Masking步骤
#DistilBert无此操作
# 如果直接将padded送过去，可能会造成混淆。
# 我们需要创造另一个参数变量来告诉他忽略mask以及我们增加的padding
attention_mask=np.where(padded!=0,1,0)
attention_mask.shape
attention_mask

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])

In [16]:
#接下来我们就可以进行深度学习了
input_ids=torch.LongTensor(padded)
# attention_mask=torch.tensor(np.array(attention_mask))


with torch.no_grad():
    last_hidden_states=model(input_ids)

运行此步骤后，last_hidden_states保存DistilBERT的输出。它是一个具有多维度的元组（示例个数，序列中的最大符号的个数，DistilBERT模型中的隐藏单元数）。在我们的例子中是2000（因为我们自行限制为2000个示例），66（这是2000个示例中最长序列中的词数量），768（DistilBERT模型中的隐藏单位数量）。

In [17]:
# 重要部分切片
# 对于句子分类问题，我们仅对[CLS]标记的BERT输出感兴趣，
# 因此我们只选择该三维数据集的一个切片。
feature=last_hidden_states[0][:,0,:].numpy()
# 现在我们获得了features这个二维numpy数组，它包含数据集中所有句子的句子嵌入。

### Logistic回归数据集
### 现在我们有了BERT的输出，已经具备训练逻辑回归模型所需的完整数据集。768列数据是特征集，而标签可以从初始数据集中获得。

我们用来训练Logistic回归的标记数据集。其中，特征是上图中切片得到的[CLS]标记（位置0）的BERT输出向量。每行对应于我们数据集中的一个句子，每列对应于Bert / DistilBERT模型顶部转换器（transformer）中前馈神经网络的隐藏单元的输出。

在完成传统的机器学习训练集、测试集划分之后，我们得到了最终的逻辑回归模型并可以对数据集进行训练了。

In [19]:
labels=df[1]
train_features, test_features, train_labels, test_labels = train_test_split(feature, labels)

In [30]:
#训练模型
lr_clf=LogisticRegression()
lr_clf.fit(train_features,train_labels)

F:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression()

In [32]:
#得到训练的模型之后进行测试评估
lr_clf.score(test_features,test_labels)

0.8341040462427746